# Module Overview

This module covers everything you need to know about parsing and ingesting data for RAG systems, from basic text files to complex PDFs and databases. We'll use LangChain v0.3 and explore each technique with practical examples.

## Table of contents
- Introduction to Data Ingestion
- Text Files(.txt)
- Markdown Files (.md)
- PDF Documents
- Microsoft Word Documents
- CSV and Excel files
- JSON and Structured Data
- Web Scraping
- Databases (SQL)
- Audio and Video Transcripts
- Advanced Techniques
- Best Practices

### Introduction To Data Ingestion

In [2]:
import os
from os.path import exists
from typing import List, Dict, Any
import pandas as pd

In [6]:
from langchain_core.documents import Document
print("Set up completed!")

Set up completed!


 ### Understanding Document Structure In LangChain

In [10]:
# Create a simple document

doc = Document(
    page_content="This is the main text content that will be embedded and searched",
    metadata={
        "source":"example.txt",
        "page": 1,
        "author": "Binod Kafle",
        "date_created": "2025-10-26"
    }
)

print("Document Structure")

print(f"Content: {doc.page_content}")
print(f"Metadata: {doc.metadata}")

# Why metadata matters:
print("Metadata is crucial for:")
print("- Filtering search results")
print("- Tracking document sources")
print("- Providing context in responses")
print("- Debugging and auditing")

Document Structure
Content: This is the main text content that will be embedded and searched
Metadata: {'source': 'example.txt', 'page': 1, 'author': 'Binod Kafle', 'date_created': '2025-10-26'}
Metadata is crucial for:
- Filtering search results
- Tracking document sources
- Providing context in responses
- Debugging and auditing


### Text Files (.txt) - The Simplest Case {#2-text-files}

In [11]:
# create a simple txt file
import os
os.makedirs("data/text_files", exist_ok=True)

In [12]:
sample_texts = {
    "data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity, readability, and versatility. Created by Guido van Rossum and first released in 1991, Python was designed to emphasize code readability with its clean syntax and use of indentation. Its philosophy focuses on making programming easier and more intuitive, allowing developers to express complex concepts in fewer lines of code compared to other languages like C++ or Java.

Python supports multiple programming paradigms, including procedural, object-oriented, and functional programming, which makes it a powerful choice for a wide range of applications. It comes with an extensive standard library and a vibrant ecosystem of third-party packages that simplify tasks such as data analysis, web development, automation, artificial intelligence, and scientific computing. Popular frameworks like Django, Flask, TensorFlow, and Pandas are built on Python, further extending its capabilities across industries.

Because of its ease of learning and wide applicability, Python has become one of the most popular programming languages in the world. It’s used by beginners learning to code, as well as professionals working in fields like web development, data science, machine learning, and cybersecurity. Python’s open-source nature, large community, and continuous development ensure that it remains a vital and evolving tool for software developers and researchers alike.""",

    "data/text_files/machine_learning.txt": """Machine Learning Basics
Machine learning (ML) is a branch of artificial intelligence (AI) that enables computers to learn and make decisions without being explicitly programmed. Instead of following strict, rule-based instructions, machine learning systems use data to identify patterns, make predictions, and improve their performance over time. The basic idea is to feed a computer a large amount of data and allow it to “learn” relationships or trends from that data through mathematical models and algorithms.

There are three main types of machine learning: supervised learning, unsupervised learning, and reinforcement learning. In supervised learning, models are trained using labeled data — for example, teaching a model to recognize cats in photos by providing many examples labeled “cat” or “not cat.” Unsupervised learning, on the other hand, deals with unlabeled data, where the algorithm tries to find hidden patterns or groupings (like clustering customers by behavior). Reinforcement learning involves training an agent through trial and error, where it learns to take actions in an environment to maximize rewards — such as a robot learning to walk or an AI playing a video game.

Machine learning has a wide range of applications in everyday life and industry. It powers technologies like voice assistants (e.g., Siri, Alexa), recommendation systems (e.g., Netflix, YouTube), fraud detection, medical diagnosis, and self-driving cars. As data becomes more abundant and computing power increases, machine learning continues to advance rapidly, playing a key role in shaping the future of automation, analytics, and decision-making across many sectors.
    """
}

for filepath, content in sample_texts.items():
    with open(filepath, 'w', encoding="utf-8") as f:
        f.write(content)

print("Sample text files created!")

Sample text files created!


### TextLoader - Read Single File


In [16]:
from langchain_community.document_loaders import TextLoader

# loading a single text file
loader = TextLoader("data/text_files/python_intro.txt", encoding="utf-8")

documents = loader.load()
print(type(documents))

print(f"Loaded {len(documents)} document")
print(f"Content preview: {documents[0].page_content[:100]}...")
print(f"Metadata: {documents[0].metadata}")


<class 'list'>
Loaded 1 document
Content preview: Python Programming Introduction

Python is a high-level, interpreted programming language known for ...
Metadata: {'source': 'data/text_files/python_intro.txt'}


### DirectoryLoader - Multiple Text Files

In [19]:
from langchain_community.document_loaders import DirectoryLoader

## load all the text files from the directory
dir_loader = DirectoryLoader(
    "data/text_files",
    glob="**/*.txt", # Pattern to match files
    loader_cls=TextLoader, ## loader class to use
    loader_kwargs={"encoding":"utf-8"},
    show_progress=True,
)

documents = dir_loader.load()

print(f" Loaded {len(documents)} documents")
for i, doc in enumerate(documents):
    print(f"\nDocument {i+1}:")
    print(f" Source: {doc.metadata["source"]}")
    print(f" Length: {len(doc.page_content)} characters")

# 📊 Analysis
print("\n📊 DirectoryLoader Characteristics:")
print("✅ Advantages:")
print("  - Loads multiple files at once")
print("  - Supports glob patterns")
print("  - Progress tracking")
print("  - Recursive directory scanning")

print("\n❌ Disadvantages:")
print("  - All files must be same type")
print("  - Limited error handling per file")
print("  - Can be memory intensive for large directories")

100%|██████████| 2/2 [00:00<00:00, 777.30it/s]

 Loaded 2 documents

Document 1:
 Source: data/text_files/python_intro.txt
 Length: 1482 characters

Document 2:
 Source: data/text_files/machine_learning.txt
 Length: 1672 characters

📊 DirectoryLoader Characteristics:
✅ Advantages:
  - Loads multiple files at once
  - Supports glob patterns
  - Progress tracking
  - Recursive directory scanning

❌ Disadvantages:
  - All files must be same type
  - Limited error handling per file
  - Can be memory intensive for large directories


### Text Splitting Strategies

In [21]:
# Different text splitting strategies
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter, TokenTextSplitter

print(documents)

[Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity, readability, and versatility. Created by Guido van Rossum and first released in 1991, Python was designed to emphasize code readability with its clean syntax and use of indentation. Its philosophy focuses on making programming easier and more intuitive, allowing developers to express complex concepts in fewer lines of code compared to other languages like C++ or Java.\n\nPython supports multiple programming paradigms, including procedural, object-oriented, and functional programming, which makes it a powerful choice for a wide range of applications. It comes with an extensive standard library and a vibrant ecosystem of third-party packages that simplify tasks such as data analysis, web development, automation, artificial intelligence, and scientific computing. Popular frameworks like Django

### Method 1 - Character Text Splitter


In [24]:
text = documents[0].page_content
text

'Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity, readability, and versatility. Created by Guido van Rossum and first released in 1991, Python was designed to emphasize code readability with its clean syntax and use of indentation. Its philosophy focuses on making programming easier and more intuitive, allowing developers to express complex concepts in fewer lines of code compared to other languages like C++ or Java.\n\nPython supports multiple programming paradigms, including procedural, object-oriented, and functional programming, which makes it a powerful choice for a wide range of applications. It comes with an extensive standard library and a vibrant ecosystem of third-party packages that simplify tasks such as data analysis, web development, automation, artificial intelligence, and scientific computing. Popular frameworks like Django, Flask, TensorFlow, and Pandas are built on Python, further extending its capab

In [25]:
# Method 1 - Character Text Splitter
print("Character Text Splitter Example")
char_splitter = CharacterTextSplitter(
    separator="\n", # Split on new lines
    chunk_size=200,  # Max chunk size in characters
    chunk_overlap=20, # Overlap between chunks
    length_function=len # How to measure chunk size
)

char_chunks = char_splitter.split_text(text)
print(f"Created {len(char_chunks)} chunks")
print(f"First chunk: {char_chunks[0][:100]}...")

Created a chunk of size 453, which is longer than the specified 200
Created a chunk of size 533, which is longer than the specified 200


Character Text Splitter Example
Created 4 chunks
First chunk: Python Programming Introduction...


In [26]:
# Method 2: Recursive character splitting (RECOMMENDED)
print("\n RECURSIVE CHARACTER SPLITTER")
recursive_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""], # Try these separators in order
    chunk_size=200,
    chunk_overlap=20,
    length_function=len
)

recursive_chunks = recursive_splitter.split_text(text)
print(f"Created {len(recursive_chunks)} chunks")
print(f"First Chunk: {recursive_chunks[0][:100]}...")


 RECURSIVE CHARACTER SPLITTER
Created 10 chunks
First Chunk: Python Programming Introduction...


In [27]:
print(recursive_chunks[0])
print("-----------------")
print(recursive_chunks[1])
print("------------------")
print(recursive_chunks[2])

Python Programming Introduction
-----------------
Python is a high-level, interpreted programming language known for its simplicity, readability, and versatility. Created by Guido van Rossum and first released in 1991, Python was designed to
------------------
was designed to emphasize code readability with its clean syntax and use of indentation. Its philosophy focuses on making programming easier and more intuitive, allowing developers to express complex


In [28]:
# Create text without natural break points
simple_text = "This is sentence one and it is quite long. This is sentence two and it is also quite long. This is sentence three which is even longer than the others. This is sentence four. This is sentence five. This is sentence six."

splitter = RecursiveCharacterTextSplitter(
    separators=[" "],  # Only split on spaces
    chunk_size=80,
    chunk_overlap=20,
    length_function=len
)

chunks = splitter.split_text(simple_text)

print(f"\nSimple text example - {len(chunks)} chunks:\n")

for i in range(len(chunks) - 1):
    print(f"Chunk {i+1}: '{chunks[i]}'")
    print(f"Chunk {i+2}: '{chunks[i+1]}'")


    print()


Simple text example - 4 chunks:

Chunk 1: 'This is sentence one and it is quite long. This is sentence two and it is also'
Chunk 2: 'two and it is also quite long. This is sentence three which is even longer than'

Chunk 2: 'two and it is also quite long. This is sentence three which is even longer than'
Chunk 3: 'is even longer than the others. This is sentence four. This is sentence five.'

Chunk 3: 'is even longer than the others. This is sentence four. This is sentence five.'
Chunk 4: 'is sentence five. This is sentence six.'



In [29]:
# Method 3: Token-based splitting
print("\n TOKEN TEXT SPLITTER")
token_splitter = TokenTextSplitter(
    chunk_size=50, # Size in tokens (not characters
    chunk_overlap=10
)

token_chunks = token_splitter.split_text(text)
print(f"Created {len(token_chunks)} chunks")
print(f"First chunk: {token_chunks[0][:100]}...")


 TOKEN TEXT SPLITTER
Created 7 chunks
First chunk: Python Programming Introduction

Python is a high-level, interpreted programming language known for ...


In [30]:
# 📊 Comparison
print("\n📊 Text Splitting Methods Comparison:")
print("\nCharacterTextSplitter:")
print("  ✅ Simple and predictable")
print("  ✅ Good for structured text")
print("  ❌ May break mid-sentence")
print("  Use when: Text has clear delimiters")

print("\nRecursiveCharacterTextSplitter:")
print("  ✅ Respects text structure")
print("  ✅ Tries multiple separators")
print("  ✅ Best general-purpose splitter")
print("  ❌ Slightly more complex")
print("  Use when: Default choice for most texts")

print("\nTokenTextSplitter:")
print("  ✅ Respects model token limits")
print("  ✅ More accurate for embeddings")
print("  ❌ Slower than character-based")
print("  Use when: Working with token-limited models")


📊 Text Splitting Methods Comparison:

CharacterTextSplitter:
  ✅ Simple and predictable
  ✅ Good for structured text
  ❌ May break mid-sentence
  Use when: Text has clear delimiters

RecursiveCharacterTextSplitter:
  ✅ Respects text structure
  ✅ Tries multiple separators
  ✅ Best general-purpose splitter
  ❌ Slightly more complex
  Use when: Default choice for most texts

TokenTextSplitter:
  ✅ Respects model token limits
  ✅ More accurate for embeddings
  ❌ Slower than character-based
  Use when: Working with token-limited models
